<a href="https://colab.research.google.com/github/i-gitit/webscrape_python/blob/master/news_summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

libraries to be installed for using this script:


*   bs4
*   requests
*   newspaper3k
*   nltk
*   time
*   pandas







In [0]:
#Installing Library
!pip3 install newspaper3k

In [0]:
#importing Libraries
from bs4 import BeautifulSoup
import requests
from newspaper import Article
import nltk
import time
nltk.download('punkt')
import pandas as pd

In [0]:
#Search function
def search_keywords(article):
  article=article.replace('\n',' ')
  for sentence in nltk.tokenize.sent_tokenize(article):
    if 'surge' in sentence or  'acquisitions' in sentence or 'IPO' in sentence:
      print("   >"+sentence)


In [0]:
#fetching html of page 
webdata=requests.get("https://news.google.com/topics/CAAqJggKIiBDQkFTRWdvSUwyMHZNRGx6TVdZU0FtVnVHZ0pKVGlnQVAB?hl=en-IN&gl=IN&ceid=IN%3Aen")
htmldata=webdata.text
soup=BeautifulSoup(htmldata,'lxml')

In [0]:
#declaring list variables
main_newstitle=[]
main_newslink=[]
main_newssummary=[]
main_newsdate=[]
sub_newstitle=[]
sub_newslink=[]
sub_newssummary=[]
sub_newsindex=[]
sub_newsdate=[]

In [0]:

a = soup.find("div", attrs={'class':'fe4pJf'})
list=a.find_all("div", attrs={'class':'NiLAwe'}) #picked news blocks, each contain main news as well as subnews

In [0]:
ln=0 #main news index used for relative indexing in subnews table
#for i in list[:15]  Can use indexing for top n blocks of news
for i in list:
  k=0 #used to identify if it is a main news or subnews
  
  for j in i.find_all('a',attrs={'class':'DY5T1d'}):
    
    url="https://news.google.com"+j['href'][1:]
    
    if(requests.get(url).status_code==200):
      
      article = Article("https://news.google.com"+j['href'][1:], language="en")
      article.download()
      article.parse() 
      article.nlp()

      #title=article.title     title using nlp
      title=j.text #Use same title as google news 
      summary=article.summary
      date=article.publish_date

      print("Title: "+title+"\nSearching keywords:")  #printing lines containing keywords.
      search_keywords(article.text)
      
      print()
      if(k==0):   #main news index
        main_newstitle.append(title)
        main_newssummary.append(summary)
        main_newslink.append(url)
        main_newsdate.append(date)
      else:       #subnews 
        sub_newstitle.append(title)
        sub_newssummary.append(summary)
        sub_newslink.append(url)
        sub_newsdate.append(date)
        sub_newsindex.append(ln)
      k+=1    
      time.sleep(3)
  ln+=1

      

In [0]:
#making tables using panda
main_news_table=pd.DataFrame({'Main news title':main_newstitle,'Summary':main_newssummary,'URL':main_newslink,'Date and Time':main_newsdate})
sub_news_table=pd.DataFrame({'Index relative to main news':sub_newsindex,'Sub news title':sub_newstitle,'Summary':sub_newssummary,'URL':sub_newslink,'Date and Time':sub_newsdate})

print("Head of News table")
main_news_table.head()


In [0]:
print("Head of Sub News table")
sub_news_table.head()